In [2]:
from google.colab import files
import io

uploaded = files.upload()

# Assuming only one file is uploaded
file_name = list(uploaded.keys())[0]
!unzip 'English to Bengali For Machine Translation.zip'


Saving English to Bengali For Machine Translation.zip to English to Bengali For Machine Translation.zip
Archive:  English to Bengali For Machine Translation.zip
  inflating: english_to_bangla.csv   


In [24]:
NUM_SENTENCES = 2000
MAX_VOCAB_SIZE = 50000
MAX_SEQUENCE_LENGTH = 200
# Define the embedding dimension
EMBEDDING_DIM = 512


# Model parameters
num_layers = 6  # Using 2 layers as specified
d_model = 128
num_heads = 8
d_ff = 2048
dropout_rate = 0.1

In [25]:

import csv
def read_text_file(filename):
    lines = []
    try:
        with open(filename, newline='\n', encoding='utf-8') as csvfile:
            csv_reader = csv.reader(csvfile)
            for row in csv_reader:
              lines.append(row)
              if len(lines) >= NUM_SENTENCES:
                break
        #print(len(lines))
        #print(lines[0])
        return lines
    except FileNotFoundError:
        return f"Error: The file '{filename}' was not found."
    except Exception as e:
        return f"An error occurred: {e}"



file_name = 'english_to_bangla.csv'
# Read the content of the file

lines = read_text_file(file_name)

import numpy as np

input_texts = []
target_texts = []
decoder_input_texts = []
decoder_target_texts = []

for line in lines:
    input_texts.append(line[0])
    target_texts.append('<sos> ' + line[1] + ' <eos>')
    decoder_input_texts.append('<sos> ' + line[1])
    decoder_target_texts.append(line[1] + ' <eos>')


In [26]:
# prompt: use tf.data.Dataset.from_generator cover  input_texts,
# decoder_input_texts
# decoder_target_texts ]

import tensorflow as tf

def data_generator():
    for i in range(len(input_texts)):
        yield (input_texts[i], decoder_input_texts[i], decoder_target_texts[i])

dataset = tf.data.Dataset.from_generator(
    data_generator,
    output_signature=(
        tf.TensorSpec(shape=(), dtype=tf.string),
        tf.TensorSpec(shape=(), dtype=tf.string),
        tf.TensorSpec(shape=(), dtype=tf.string)
    )
)

# Example of how to iterate over the dataset
#for input_text, dec_input_text, dec_target_text in dataset.take(5):
#    print(f"Input: {input_text.numpy().decode('utf-8')}")
#    print(f"Decoder Input: {dec_input_text.numpy().decode('utf-8')}")
#    print(f"Decoder Target: {dec_target_text.numpy().decode('utf-8')}")


In [27]:
# prompt: TextVectorization on dataset object

#MAX_SEQUENCE_LENGTH = max(max(len(input_vectorization(x).numpy()), len(target_vectorization(y).numpy())) for x, y, z in dataset.take(NUM_SENTENCES))

# Input Text Vectorization
input_vectorization = tf.keras.layers.TextVectorization(
    max_tokens=MAX_VOCAB_SIZE,
    standardize='lower_and_strip_punctuation',
    output_sequence_length=MAX_SEQUENCE_LENGTH # Set this based on the desired sequence length
)

# Build the vocabulary for the input texts from the dataset
input_dataset_for_vocab = dataset.map(lambda x, y, z: x)
input_vectorization.adapt(input_dataset_for_vocab.batch(64)) # Batching is important for adapt

# Target Text Vectorization
target_vectorization = tf.keras.layers.TextVectorization(
    max_tokens=MAX_VOCAB_SIZE,
    standardize='lower_and_strip_punctuation', # Can adjust standardization for target if needed
    output_sequence_length=MAX_SEQUENCE_LENGTH # Set this based on the desired sequence length
)

# Build the vocabulary for the target texts from the dataset
# We use decoder_input_texts and include '<eos>' to build the vocabulary
target_dataset_for_vocab = dataset.map(lambda x, y, z: y).concatenate(tf.data.Dataset.from_tensor_slices(['<eos>']))
target_vectorization.adapt(target_dataset_for_vocab.batch(64)) # Batching is important for adapt


# Apply vectorization to the dataset
def vectorize_texts(input_text, decoder_input_text, decoder_target_text):
    input_vectorized = input_vectorization(input_text)
    decoder_input_vectorized = target_vectorization(decoder_input_text)
    decoder_target_vectorized = target_vectorization(decoder_target_text)
    return (input_vectorized, decoder_input_vectorized), decoder_target_vectorized

vectorized_dataset = dataset.map(vectorize_texts)

# Example of how to iterate over the vectorized dataset
#for input_vec, dec_input_vec, dec_target_vec in vectorized_dataset.take(5):
#    print(f"Input Vectorized: {input_vec.numpy()}")
#    print(f"Decoder Input Vectorized: {dec_input_vec.numpy()}")
#    print(f"Decoder Target Vectorized: {dec_target_vec.numpy()}")

In [28]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import Layer, Dense, Embedding, Dropout, LayerNormalization, MultiHeadAttention
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import math

In [29]:
# prompt: split the vectorized_dataset into training and validaation with 80% for trining

dataset_size = NUM_SENTENCES #tf.data.experimental.cardinality(vectorized_dataset).numpy()
train_size = int(0.8 * dataset_size)
val_size = dataset_size - train_size

train_dataset = vectorized_dataset.take(train_size)
validation_dataset = vectorized_dataset.skip(train_size).take(val_size)

# Optional: Batch the datasets for training
BATCH_SIZE = 32
train_dataset = train_dataset.batch(BATCH_SIZE, drop_remainder=True) # Add drop_remainder=True
validation_dataset = validation_dataset.batch(BATCH_SIZE, drop_remainder=True) # Add drop_remainder=True

# Optional: Prefetch data for performance
AUTOTUNE = tf.data.AUTOTUNE
train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.prefetch(buffer_size=AUTOTUNE)

print(f"Total dataset size: {dataset_size}")
# Calculate the actual number of samples after batching with drop_remainder=True
actual_train_size = (train_size // BATCH_SIZE) * BATCH_SIZE
actual_val_size = (val_size // BATCH_SIZE) * BATCH_SIZE

print(f"Training dataset size: {actual_train_size}")
print(f"Validation dataset size: {actual_val_size}")

Total dataset size: 2000
Training dataset size: 1600
Validation dataset size: 384


In [30]:
# prompt: add a embedding layer of 128 for the input length of MAX_SEQUENCE_LENGTH

# Get the vocabulary size from the vectorization layer
input_vocab_size = input_vectorization.vocabulary_size()


# Create the embedding layer
encoder_embedding_layer = tf.keras.layers.Embedding(
    input_vocab_size,
    EMBEDDING_DIM,
    input_length=MAX_SEQUENCE_LENGTH
)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [31]:
# Positional Encoding Layer
class PositionalEncoding(Layer):
    def __init__(self, max_seq_len, d_model):
        super(PositionalEncoding, self).__init__()
        self.d_model = d_model
        self.max_seq_len = max_seq_len

        # Create positional encoding matrix in __init__
        pos_encoding = np.zeros((self.max_seq_len, self.d_model))
        for pos in range(self.max_seq_len):
            for i in range(0, self.d_model, 2):
                pos_encoding[pos, i] = math.sin(pos / (10000 ** ((2 * i) / self.d_model)))
                if i + 1 < self.d_model:
                    pos_encoding[pos, i + 1] = math.cos(pos / (10000 ** ((2 * i) / self.d_model)))

        # Store as a non-trainable TensorFlow constant
        self.pos_encoding = tf.constant(pos_encoding, dtype=tf.float32)


    def call(self, inputs):
        seq_len = tf.shape(inputs)[1]
        # Ensure the positional encoding is added correctly by slicing
        # Correct slicing for 2D pos_encoding: [sequence_length, embedding_dimension]
        return inputs + self.pos_encoding[:seq_len, :]

In [32]:
# Feed Forward Network
class FeedForwardNetwork(Layer):
    def __init__(self, d_model, d_ff, dropout_rate=0.1):
        super(FeedForwardNetwork, self).__init__()
        self.d_model = d_model
        self.d_ff = d_ff
        self.dropout_rate = dropout_rate

    def build(self, input_shape):
        self.dense1 = Dense(self.d_ff, activation='relu')
        self.dense2 = Dense(self.d_model)
        self.dropout = Dropout(self.dropout_rate)

    def call(self, inputs, training=None):
        x = self.dense1(inputs)
        x = self.dropout(x, training=training)
        return self.dense2(x)

In [33]:
# Encoder Layer
class EncoderLayer(Layer):
    def __init__(self, d_model, num_heads, d_ff, dropout_rate=0.1):
        super(EncoderLayer, self).__init__()
        self.d_model = d_model
        self.num_heads = num_heads
        self.d_ff = d_ff
        self.dropout_rate = dropout_rate

    def build(self, input_shape):
        # Multi-Head Self-Attention
        self.mha = MultiHeadAttention(num_heads=self.num_heads, key_dim=self.d_model)
        self.dropout1 = Dropout(self.dropout_rate)
        self.layernorm1 = LayerNormalization(epsilon=1e-6)

        # Feed Forward Network
        self.ffn = FeedForwardNetwork(self.d_model, self.d_ff, self.dropout_rate)
        self.dropout2 = Dropout(self.dropout_rate)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)

    def call(self, inputs, training=None, mask=None):
        # Self-attention with residual connection and layer norm
        attn_output = self.mha(inputs, inputs, attention_mask=mask)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)

        # Feed forward with residual connection and layer norm
        ffn_output = self.ffn(out1, training=training)
        ffn_output = self.dropout2(ffn_output, training=training)
        out2 = self.layernorm2(out1 + ffn_output)

        return out2


In [34]:
# Encoder
class Encoder(Layer):
    def __init__(self, num_layers, d_model, num_heads, d_ff, input_vocab_size, max_seq_len, dropout_rate=0.1):
        super(Encoder, self).__init__()
        self.d_model = d_model
        self.num_layers = num_layers

        self.embedding = Embedding(input_vocab_size, d_model)
        self.pos_encoding = PositionalEncoding(max_seq_len, d_model)
        self.dropout = Dropout(dropout_rate)

        self.enc_layers = [EncoderLayer(d_model, num_heads, d_ff, dropout_rate)
                          for _ in range(num_layers)]

    def call(self, inputs, training=None, mask=None):
        # Embedding and positional encoding
        x = self.embedding(inputs)
        x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32)) ##
        x = self.pos_encoding(x)
        x = self.dropout(x, training=training)

        # Pass through encoder layers
        for i in range(self.num_layers):
            x = self.enc_layers[i](x, training=training, mask=mask)

        return x


In [35]:
# Decoder Layer
class DecoderLayer(Layer):
    def __init__(self, d_model, num_heads, d_ff, dropout_rate=0.1):
        super(DecoderLayer, self).__init__()
        self.d_model = d_model
        self.num_heads = num_heads
        self.d_ff = d_ff
        self.dropout_rate = dropout_rate

    def build(self, input_shape):
        # Masked Multi-Head Self-Attention
        self.mha1 = MultiHeadAttention(num_heads=self.num_heads, key_dim=self.d_model)
        self.dropout1 = Dropout(self.dropout_rate)
        self.layernorm1 = LayerNormalization(epsilon=1e-6)

        # Multi-Head Cross-Attention
        self.mha2 = MultiHeadAttention(num_heads=self.num_heads, key_dim=self.d_model)
        self.dropout2 = Dropout(self.dropout_rate)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)

        # Feed Forward Network
        self.ffn = FeedForwardNetwork(self.d_model, self.d_ff, self.dropout_rate)
        self.dropout3 = Dropout(self.dropout_rate)
        self.layernorm3 = LayerNormalization(epsilon=1e-6)

    def call(self, inputs, encoder_output, training=None, look_ahead_mask=None, padding_mask=None):
        # Masked self-attention with residual connection and layer norm
        attn1 = self.mha1(inputs, inputs, attention_mask=look_ahead_mask)
        attn1 = self.dropout1(attn1, training=training)
        out1 = self.layernorm1(inputs + attn1)

        # Cross-attention with residual connection and layer norm
        attn2 = self.mha2(out1, encoder_output, attention_mask=padding_mask)
        attn2 = self.dropout2(attn2, training=training)
        out2 = self.layernorm2(out1 + attn2)

        # Feed forward with residual connection and layer norm
        ffn_output = self.ffn(out2, training=training)
        ffn_output = self.dropout3(ffn_output, training=training)
        out3 = self.layernorm3(out2 + ffn_output)

        return out3

In [36]:
# Decoder
class Decoder(Layer):
    def __init__(self, num_layers, d_model, num_heads, d_ff, target_vocab_size, max_seq_len, dropout_rate=0.1):
        super(Decoder, self).__init__()
        self.d_model = d_model
        self.num_layers = num_layers

        self.embedding = Embedding(target_vocab_size, d_model)
        self.pos_encoding = PositionalEncoding(max_seq_len, d_model)
        self.dropout = Dropout(dropout_rate)

        self.dec_layers = [DecoderLayer(d_model, num_heads, d_ff, dropout_rate)
                          for _ in range(num_layers)]

    def call(self, inputs, encoder_output, training=None, look_ahead_mask=None, padding_mask=None):
        # Embedding and positional encoding
        x = self.embedding(inputs)
        x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        x = self.pos_encoding(x)
        x = self.dropout(x, training=training)

        # Pass through decoder layers
        for i in range(self.num_layers):
            x = self.dec_layers[i](x, encoder_output, training=training,
                                 look_ahead_mask=look_ahead_mask, padding_mask=padding_mask)

        return x


In [37]:

# Complete Transformer Model
class Transformer(Model):
    def __init__(self, num_layers, d_model, num_heads, d_ff, input_vocab_size,
                 target_vocab_size, max_seq_len, dropout_rate=0.1):
        super(Transformer, self).__init__()

        # Use 2 encoder layers and 2 decoder layers as specified
        self.encoder = Encoder(2, d_model, num_heads, d_ff, input_vocab_size,
                              max_seq_len, dropout_rate)
        self.decoder = Decoder(2, d_model, num_heads, d_ff, target_vocab_size,
                              max_seq_len, dropout_rate)

        # Final dense layer and softmax
        self.final_layer = Dense(target_vocab_size)

    def call(self, inputs, training=None):

        inp, tar = inputs

        # Create masks
        enc_padding_mask = self.create_padding_mask(inp)
        dec_padding_mask = self.create_padding_mask(inp)
        look_ahead_mask = self.create_look_ahead_mask(tf.shape(tar)[1])
        dec_target_padding_mask = self.create_padding_mask(tar)
        combined_mask = tf.maximum(dec_target_padding_mask, look_ahead_mask)

        # Encoder output
        enc_output = self.encoder(inp, training=training, mask=enc_padding_mask)

        # Decoder output
        dec_output = self.decoder(tar, enc_output, training=training,
                                look_ahead_mask=combined_mask,
                                padding_mask=dec_padding_mask)

        # Final linear layer with softmax
        final_output = self.final_layer(dec_output)

        return final_output

    def create_padding_mask(self, seq):
        """
        Creates a mask to hide padding tokens (zeros) in the sequence.

        Args:
            seq: Input sequence tensor of shape (batch_size, seq_len)

        Returns:
            Mask tensor of shape (batch_size, 1, 1, seq_len) where:
            - 1.0 = mask this position (it's a padding token)
            - 0.0 = don't mask this position (it's a real token)
        """
        # Step 1: Find where the sequence has padding tokens (value = 0)
        # This creates True where seq == 0, False elsewhere
        is_padding = tf.math.equal(seq, 0)

        # Step 2: Convert boolean to float (True->1.0, False->0.0)
        mask = tf.cast(is_padding, tf.float32)

        # Step 3: Add extra dimensions for broadcasting with attention weights
        # Shape changes from (batch_size, seq_len) to (batch_size, 1, 1, seq_len)
        mask = mask[:, tf.newaxis, tf.newaxis, :]

        return mask

    def create_look_ahead_mask(self, size):
        """
        Creates a triangular mask to prevent the decoder from seeing future tokens.
        This ensures that when predicting token at position i, we can only see
        tokens at positions 0, 1, 2, ..., i-1.

        Args:
            size: Length of the sequence

        Returns:
            Mask tensor of shape (size, size) where:
            - 1.0 = mask this position (future token, shouldn't be seen)
            - 0.0 = don't mask this position (past/current token, can be seen)

        Example for size=4:
        [[0, 1, 1, 1],    # Position 0 can only see itself
         [0, 0, 1, 1],    # Position 1 can see positions 0,1
         [0, 0, 0, 1],    # Position 2 can see positions 0,1,2
         [0, 0, 0, 0]]    # Position 3 can see positions 0,1,2,3
        """
        # Step 1: Create a matrix of all ones
        ones_matrix = tf.ones((size, size))

        # Step 2: Create lower triangular matrix (including diagonal)
        # band_part(matrix, -1, 0) keeps lower triangle + diagonal
        lower_triangular = tf.linalg.band_part(ones_matrix, -1, 0)

        # Step 3: Subtract from 1 to flip the mask
        # Lower triangle becomes 0 (don't mask), upper triangle becomes 1 (mask)
        look_ahead_mask = 1 - lower_triangular

        return look_ahead_mask

In [47]:
# Training Setup
def create_model(input_vocab_size, target_vocab_size, max_seq_len): # Removed default value
    # Create model
    transformer = Transformer(
        num_layers=num_layers,
        d_model=d_model,
        num_heads=num_heads,
        d_ff=d_ff,
        input_vocab_size=input_vocab_size,
        target_vocab_size=target_vocab_size,
        max_seq_len=max_seq_len,
        dropout_rate=dropout_rate
    )

    return transformer

In [48]:
# Updated Inference Model
class TranslationInference:
    def __init__(self, trained_model, input_vectorizer, target_vectorizer, max_length=100):
        self.model = trained_model
        self.input_vectorizer = input_vectorizer
        self.target_vectorizer = target_vectorizer
        self.max_length = max_length

        # Get vocabularies
        self.input_vocab = input_vectorizer.get_vocabulary()
        self.target_vocab = target_vectorizer.get_vocabulary()

        # Create reverse vocabulary for target language
        self.target_index_lookup = {i: token for i, token in enumerate(self.target_vocab)}

        # Get special tokens
        self.start_token_id = self.target_vectorizer(['<sos>'])[0][0].numpy()
        self.end_token_id = self.target_vectorizer(['<eos>'])[0][0].numpy()

    def translate(self, sentence):
        # Tokenize input sentence
        inputs = self.input_vectorizer([sentence])
        inputs = tf.expand_dims(inputs[0], 0)

        # Initialize decoder input with start token
        decoder_input = tf.expand_dims([self.start_token_id], 0)

        for i in range(self.max_length):
            # Get predictions
            predictions = self.model([inputs, decoder_input], training=False)

            # Select the last token from the seq_len dimension
            predictions = predictions[:, -1:, :]
            predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)

            # Concatenate the predicted_id to the output
            decoder_input = tf.concat([decoder_input, predicted_id], axis=-1)

            # Check for end token
            if predicted_id[0, 0] == self.end_token_id:
                break

        # Convert tokens to text
        result = decoder_input[0].numpy()
        predicted_tokens = [self.target_index_lookup.get(token_id, '<UNK>')
                           for token_id in result[1:]]  # Skip start token

        # Remove end token if present and join
        if '<eos>' in predicted_tokens:
            predicted_tokens = predicted_tokens[:predicted_tokens.index('<eos>')]

        predicted_sentence = ' '.join(predicted_tokens)
        return predicted_sentence


In [49]:
# Training function
def train_model(model, train_dataset, val_dataset, epochs=10):
    # Compile model with standard loss (no need for custom masked loss)
    model.compile(
        optimizer=Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.98, epsilon=1e-9),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=['accuracy']
    )

    # Callbacks
    callbacks = [
        tf.keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True),
        tf.keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=2)
    ]

    # Train the model
    history = model.fit(
        train_dataset,
        validation_data=val_dataset,
        epochs=epochs,
        callbacks=callbacks,
        verbose=1
    )

    return history

In [50]:
input_vocab_size = len(input_vectorization.get_vocabulary())
target_vocab_size = len(target_vectorization.get_vocabulary())

print(f"Input vocabulary size: {input_vocab_size}")
print(f"Target vocabulary size: {target_vocab_size}")


Input vocabulary size: 1913
Target vocabulary size: 3094


In [51]:
model = create_model(input_vocab_size, target_vocab_size, MAX_SEQUENCE_LENGTH)

In [ ]:
history = train_model(model, train_dataset, validation_dataset, epochs=1)

      9/Unknown 139s 12s/step - accuracy: 0.6305 - loss: 6.6793

In [ ]:
model.summary()

In [ ]:
from tensorflow.keras.utils import plot_model
plot_model(model, to_file='model_architecture.png',
           show_shapes=True, show_layer_names=True, show_layer_activations=True,
           show_dtype=True,  expand_nested=True, show_trainable=False)


In [ ]:
inference = TranslationInference(model, input_vectorization, target_vectorization)

english_sentences = [
    "Hello, how are you?",
    "I love programming.",
    "The weather is beautiful today.",
    "Thank you for your help.",
    "What is your name?"
]

print("\nTranslation Examples:")
print("=" * 50)
for sentence in english_sentences:
    try:
        translation = inference.translate(sentence)
        print(f"English: {sentence}")
        print(f"Bengali: {translation}")
        print("-" * 30)
    except Exception as e:
        print(f"Error translating '{sentence}': {e}")